#### Contextual compressing retriever in langchain is an advanced retriever that improves retrieval quality by compressing documents after retrival -- keeping only relevant content based on the users query

In [1]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI, OpenAIEmbeddings 
from langchain_core.documents import Document
from langchain.retrievers.document_compressors import LLMChainExtractor

In [2]:
# document 
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [3]:
# Embedding model
embedding_model = OpenAIEmbeddings()

# Vector Store 
vector_store = FAISS.from_documents(
    docs, embedding_model
)

In [9]:
base_retriever = vector_store.as_retriever(search_kwargs = {'k': 5})

In [10]:
# Compressor using LLM 
llm = ChatOpenAI(model = "gpt-3.5-turbo")

compressor = LLMChainExtractor.from_llm(llm)

In [11]:
# Contextual compression retriever 
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever, 
    base_compressor=compressor
)

In [12]:
query = "What is photosynthesis ?"
compressed_result = compression_retriever.invoke(query)

In [14]:
for i, doc in enumerate(compressed_result):
    print(f"\n=== Result {i + 1} ===")
    print(doc.page_content)


=== Result 1 ===
Photosynthesis is the process by which green plants convert sunlight into energy.

=== Result 2 ===
The chlorophyll in plant cells captures sunlight during photosynthesis.
